In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
import os

os.environ['ORCHESTRA_DEPLOYMENT_TYPE'] = 'remote'
os.environ['DEVSPACE_PROFILE'] = 'bigquery-scenario-tests'
os.environ['GITHUB_CI'] = 'false'
os.environ['SYFT_BASE_IMAGE_REGISTRY'] = 'k3d-registry.localhost:5800'
os.environ['DATASITE_CLUSTER_NAME'] = 'test-datasite-1'
os.environ['SERVER_URL'] = 'http://localhost'
os.environ['SERVER_PORT'] = '9081'
os.environ['TEST_EXTERNAL_REGISTRY'] = 'k3d-registry.localhost:5800'
os.environ['TEST_QUERY_LIMIT_SIZE'] = '500000'
os.environ['TRACING'] = 'False'
os.environ['NUM_TEST_USERS'] = '5'
os.environ['NUM_TEST_JOBS'] = '10'
os.environ['NUM_TEST_WORKERS'] = '2'

In [ ]:
# third party
from helpers import get_email_server
from helpers import load_users
from job_helpers import load_jobs
from job_helpers import save_jobs

# syft absolute
import syft as sy

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

# Launch server & login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="9081",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:9081", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
email_server, smtp_server = get_email_server()

# Download results

In [ ]:
users = load_users(high_client)
jobs = load_jobs(users, high_client)

In [ ]:
# submitted_jobs = [job for job in jobs if job.is_submitted]
reviewed_jobs = [job for job in jobs if job.admin_reviewed]
reviewed_jobs_should_succeed = [j for j in reviewed_jobs if j.should_succeed]
reviewed_jobs_should_fail = [j for j in reviewed_jobs if not j.should_succeed]

print(
    f"{len(reviewed_jobs)=}, {len(reviewed_jobs_should_succeed)=}, {len(reviewed_jobs_should_fail)=}"
)

In [ ]:
# TODO: test jobs that were never approved
# they seem to give weird errors like
# "You uploaded an ActionObject that is not yet in the blob storage"

In [ ]:
for job in reviewed_jobs_should_succeed:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method
    j = api_method(blocking=False)
    res = j.wait()

    if isinstance(res, sy.SyftError):
        raise sy.SyftException(public_message="Expected success, got error")

    result = res.get()
    job.result_as_expected = True

In [ ]:
for job in reviewed_jobs_should_fail:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method

    j = api_method(blocking=False)
    res = j.wait()
    if isinstance(res, sy.SyftError):
        job.result_as_expected = True
    else:
        raise sy.SyftException(public_message=f"failed, job didnt raise {type(j)}")

In [ ]:
save_jobs(jobs)

In [ ]:
# TODO fix
expected_jobs = [job for job in jobs if job.result_as_expected]
print(f"got expected_jobs: {len(expected_jobs)} == reviewed_jobs: {len(reviewed_jobs)}")
assert len(reviewed_jobs) == len(expected_jobs)

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()